# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.137562
std,1.740254
min,0.004530
25%,0.010635
50%,0.016778
75%,0.032392
max,46.419552


In [3]:
# plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=10)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.019962
std,0.013149
min,0.004530
25%,0.010309
50%,0.014975
75%,0.026453
max,0.067472


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 9.3906e-01   valLoss:9.6108e-01  time: 1.75e+00
epoch: 1   trainLoss: 7.7934e-01   valLoss:9.5168e-01  time: 1.46e+00
epoch: 2   trainLoss: 6.4415e-01   valLoss:9.7877e-01  time: 1.43e+00
epoch: 3   trainLoss: 5.1322e-01   valLoss:1.0715e+00  time: 1.43e+00
epoch: 4   trainLoss: 4.4452e-01   valLoss:1.3035e+00  time: 1.42e+00
epoch: 5   trainLoss: 3.9518e-01   valLoss:1.8738e+00  time: 1.40e+00
epoch: 6   trainLoss: 3.5037e-01   valLoss:2.5668e+00  time: 1.40e+00
epoch: 7   trainLoss: 2.9780e-01   valLoss:2.6705e+00  time: 1.44e+00
epoch: 8   trainLoss: 2.3464e-01   valLoss:1.4771e+00  time: 1.45e+00
epoch: 9   trainLoss: 2.3480e-01   valLoss:5.1360e-01  time: 1.44e+00
epoch: 10   trainLoss: 2.2997e-01   valLoss:3.6351e-01  time: 1.45e+00
epoch: 11   trainLoss: 2.3081e-01   valLoss:4.8568e-01  time: 1.44e+00
epoch: 12   trainLoss: 1.8157e-01   valLoss:5.6041e-01  time: 1.41e+00
epoch: 13   trainLoss: 2.2463e-01   valLoss:3.3168e-01  time: 1.41e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000005,0.001412,0.074794,0.917876,0.972318,0.843597,0.0
test,0.000007,0.001558,0.083019,0.821941,0.938668,0.807591,0.0


## 5. Visualize some predictions

In [7]:
i = 4
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=20, prediction=pred)

UnboundLocalError: local variable 'height' referenced before assignment

In [ ]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');